# In this example we'll be focused on automated feature engineering
## We will take the same example files, turn them into a `sqlite` 
## database and perform automated feature engineering relative
## to a selected target variable.

In [17]:
import pandas as pd
import sqlite3
import json
import os
import typing
import datetime


# examples for using SQL engines and dialects
from graphreduce.node import SQLNode
from graphreduce.graph_reduce import GraphReduce
from graphreduce.enum import SQLOpType, ComputeLayerEnum, PeriodUnit
from graphreduce.models import sqlop
from graphreduce.context import method_requires

In [18]:
!ls dat/

cust.csv                      order_events.csv
cust.db                       order_products.csv
notification_interactions.csv orders.csv
notifications.csv             products.csv


In [19]:
!rm dat/cust.db

In [20]:
dbfile = 'dat/cust.db'
conn = sqlite3.connect(dbfile)

In [21]:
files = [x for x in os.listdir('dat/') if x.endswith('.csv')]
# add each file to the database
for f in files:
    df = pd.read_csv(f"dat/{f}")
    name = f.split('.')[0]
    df.to_sql(name, conn, if_exists='replace', index=False)

In [22]:
cust = SQLNode(fpath='cust',
                prefix='cust',
                client=conn, 
                compute_layer=ComputeLayerEnum.sqlite, 
                columns=['id','name'])
order = SQLNode(fpath='orders',
               prefix='ord',
               client=conn,
               compute_layer=ComputeLayerEnum.sqlite,
               columns=['id','customer_id','ts','amount'],
                date_key='ts')

2024-07-05 16:20:39 [warning  ] no `date_key` set for <GraphReduceNode: fpath=cust fmt=>


In [23]:
gr = GraphReduce(
    name='sql_autofe',
    parent_node=cust,
    # Cut date for filtering.
    cut_date=datetime.datetime(2023, 7, 3),
    # Feature parameters.
    compute_period_unit=PeriodUnit.day,
    compute_period_val=365,
    # Label parameters.
    label_node=order,
    label_field='amount',
    label_operation='sum',
    label_period_unit=PeriodUnit.day,
    label_period_val=30,
    compute_layer=ComputeLayerEnum.sqlite,
    use_temp_tables=True,
    lazy_execution=False,
    
    # Auto feature engineering params.
    auto_features=True,
    auto_feature_hops_back=2,
    auto_feature_hops_front=1
)


In [24]:
gr.add_node(cust)
gr.add_node(order)
gr.add_entity_edge(
    cust,
    order,
    parent_key='id',
    relation_key='customer_id',
    reduce=True
)

In [25]:
gr.do_transformations_sql()

2024-07-05 16:20:40 [info     ] hydrating graph attributes
2024-07-05 16:20:40 [info     ] hydrating attributes for SQLNode
2024-07-05 16:20:40 [info     ] hydrating attributes for SQLNode
2024-07-05 16:20:40 [info     ] checking for prefix uniqueness
2024-07-05 16:20:40 [info     ] no sql was provided for do_annotate so using current data ref
2024-07-05 16:20:40 [info     ] no sql was provided for do_filters so using current data ref
2024-07-05 16:20:40 [info     ] no sql was provided for do_normalize so using current data ref
2024-07-05 16:20:40 [info     ] no sql was provided for do_annotate so using current data ref
2024-07-05 16:20:40 [info     ] no sql was provided for do_filters so using current data ref
2024-07-05 16:20:40 [info     ] no sql was provided for do_normalize so using current data ref
2024-07-05 16:20:40 [info     ] depth-first traversal through the graph from source: <GraphReduceNode: fpath=cust fmt=>
2024-07-05 16:20:40 [info     ] reducing relation <GraphReduceNo

In [26]:
gr.parent_node._temp_refs

{'do_data': 'SQLNode_cust_do_data',
 'do_annotate': 'SQLNode_cust_do_data',
 'do_filters': 'SQLNode_cust_do_data',
 'do_normalize': 'SQLNode_cust_do_data',
 'join': 'SQLNode_cust_join1',
 'do_post_join_annotate': 'SQLNode_cust_join1',
 'do_post_join_filters': 'SQLNode_cust_join1'}

In [27]:
order._temp_refs

{'do_data': 'SQLNode_orders_do_data',
 'do_annotate': 'SQLNode_orders_do_data',
 'do_filters': 'SQLNode_orders_do_data',
 'do_normalize': 'SQLNode_orders_do_data',
 'do_reduce': 'SQLNode_orders_do_reduce',
 'do_labels': 'SQLNode_orders_do_labels'}

In [28]:
gr.feature_typefunc_map

{'int64': ['count'],
 'str': ['min', 'max', 'count'],
 'object': ['count'],
 'float64': ['min', 'max', 'sum'],
 'datetime64': ['min', 'max'],
 'datetime64[ns]': ['min', 'max']}

In [29]:
order._temp_refs

{'do_data': 'SQLNode_orders_do_data',
 'do_annotate': 'SQLNode_orders_do_data',
 'do_filters': 'SQLNode_orders_do_data',
 'do_normalize': 'SQLNode_orders_do_data',
 'do_reduce': 'SQLNode_orders_do_reduce',
 'do_labels': 'SQLNode_orders_do_labels'}

In [30]:
pd.read_sql_query("SELECT name FROM sqlite_temp_master WHERE type='table'", conn)

,name
0,SQLNode_cust_do_data
1,SQLNode_orders_do_data
2,SQLNode_orders_do_reduce
3,SQLNode_cust_join
4,SQLNode_orders_do_labels
5,SQLNode_cust_join1


In [31]:
pd.read_sql_query("select * from SQLNode_cust_join1", conn)

,cust_id,cust_name,ord_customer_id,ord_id_count,ord_customer_id_count,ord_ts_count,ord_amount_min,ord_amount_max,ord_amount_sum,ord_customer_id:1,ord_amount_label
0,1,wes,1,3,3,3,10.0,325.0,346.5,1.0,12000.0
1,2,john,2,3,3,3,23.0,150.0,273.0,NaN,NaN


In [32]:
pd.read_sql_query("select * from SQLNode_orders_do_data", conn)

,ord_id,ord_customer_id,ord_ts,ord_amount
0,1,1,2023-05-12,10.0
1,2,1,2023-06-01,11.5
2,3,2,2023-01-01,100.0
3,4,2,2022-08-05,150.0
4,5,1,2023-07-01,325.0
5,6,2,2023-07-02,23.0
6,7,1,2023-07-14,12000.0
